## Using BoM data to present 2017 hourly information on temperatures and precipitations ##

Step 1 : Importing the libraries and functions saved in a py file

In [1]:
import pandas as pd
import numpy as np
import glob
import datetime 
import math 
import time

from Weather_functions import Datastations
from Weather_functions import Datagap
from Weather_functions import Datafilling


Step 2 : Uploading data from the weather stations

In [2]:
start_time = time.time() #Check running time of data upload (i would like to improve that running time in the future)

station = int(input("station number? ")) #68241 = Dapto

df1 = Datastations(station,"refdata/obs") # Run function to extract and combine csv files for 2017/2018
df2 = Datastations(station,"refdata/BoM_ETA_20160501-20170430/obs") # Run function to extract and combine csv files for 2016/2017

print (len(df1)) 
print (df1.head()) 
print (len(df2)) 
print (df2.head()) 

print("--- %s seconds ---" % (time.time() - start_time))


station number? 68241
33519
   station_number  area_code parameter  valid_start   valid_end  value  \
0           68241  NSW_PT001      MaxT   1493596800  1493600400   22.1   
1           68241  NSW_PT001      MaxT   1493600400  1493604000   23.5   
2           68241  NSW_PT001      MaxT   1493604000  1493607600   24.1   
3           68241  NSW_PT001      MaxT   1493607600  1493611200   24.7   
4           68241  NSW_PT001      MaxT   1493611200  1493614800   24.5   

      unit statistic level  qc_valid_minutes  qc_valid_start  qc_valid_end  
0  Celsius       max   SFC                60      1493596800    1493600400  
1  Celsius       max   SFC                57      1493600400    1493604000  
2  Celsius       max   SFC                60      1493604000    1493607600  
3  Celsius       max   SFC                60      1493607600    1493611200  
4  Celsius       max   SFC                60      1493611200    1493614800  
35036
   station_number  area_code parameter  valid_start   valid

Step 3 : Changing time format

In [3]:
#for df1 

df1["valid_start"] = df1["valid_start"].apply(pd.to_numeric)
df1["valid_end"] = df1["valid_end"].apply(pd.to_numeric)

df1["valid_start"]= df1["valid_start"]+36000 # add 10h to convert GMT to Australia time
df1["valid_end"]= df1["valid_end"]+36000

df1["valid_start"] = pd.to_datetime(df1["valid_start"],unit='s') # convert epoch time in valid time
df1["valid_end"] = pd.to_datetime(df1["valid_end"],unit='s') 

print(df1['valid_start'].values[1],df1['valid_end'].values[1])

#for df2

df2["valid_start"] = df2["valid_start"].apply(pd.to_numeric)
df2["valid_end"] = df2["valid_end"].apply(pd.to_numeric)

df2["valid_start"]= df2["valid_start"]+36000 # add 10h to convert GMT to Australia time
df2["valid_end"]= df2["valid_end"]+36000

df2.loc[df2.parameter =="AIR_TEMP", "valid_end"] = df2["valid_end"]+3600 # add 1h to AIR_TEMP (appears as instantaneous)
df2.loc[df2.parameter =="PRCP", "valid_end"] = df2["valid_end"]+3000 # Precip. are only reported on the first 10 min of each hour so add 50 min.
 
df2["valid_start"] = pd.to_datetime(df2["valid_start"],unit='s') #convert epoch time in normal time
df2["valid_end"] = pd.to_datetime(df2["valid_end"],unit='s') 

print(df2['valid_start'].values[1],df2['valid_end'].values[1])

2017-05-01T11:00:00.000000000 2017-05-01T12:00:00.000000000
2016-05-01T01:00:00.000000000 2016-05-01T02:00:00.000000000


step 4 : Data cleaning 

In [4]:
#for df1
df1['T_Celsius'] = np.where(df1['parameter']=='T', df1['value'], '') # create new columns before dropping unncessary ones
df1['MinT_Celsius'] = np.where(df1['parameter']=='MinT', df1['value'], '')
df1['MaxT_Celsius'] = np.where(df1['parameter']=='MaxT', df1['value'], '')
df1['Precip_mm'] = np.where(df1['parameter']=='Precip', df1['value'], '')

df1= df1.drop(["area_code", "unit", "statistic", "level","qc_valid_minutes","parameter","value","qc_valid_start","qc_valid_end"], axis=1) # drop unncessary columns 
df1 = df1.groupby(['valid_start','valid_end','station_number'])['T_Celsius','MinT_Celsius','MaxT_Celsius','Precip_mm'].sum().reset_index()

print (len(df1)) #-> result should be close to (365*24=8760)
print(df1.head())

#for df2 ->repeat same steps 

df2['T_Celsius'] = np.where(df2['parameter']=='AIR_TEMP', df2['value'], '')
df2['MinT_Celsius'] = np.where(df2['parameter']=='AIR_TEMP_MIN', df2['value'], '')
df2['MaxT_Celsius'] = np.where(df2['parameter']=='AIR_TEMP_MAX', df2['value'], '')
df2['Precip_mm'] = np.where(df2['parameter']=='PRCP', df2['value'], '')

df2= df2.drop(["area_code", "unit", "statistic", "level","qc_valid_minutes","parameter","value","instantaneous","qc_valid_minutes_start","qc_valid_minutes_end"], axis=1) 
df2 = df2.groupby(['valid_start','valid_end','station_number'])['T_Celsius','MinT_Celsius','MaxT_Celsius','Precip_mm'].sum().reset_index()

print (len(df2)) 
print(df2.head())



8563
          valid_start           valid_end  station_number T_Celsius  \
0 2017-05-01 10:00:00 2017-05-01 11:00:00           68241      20.8   
1 2017-05-01 11:00:00 2017-05-01 12:00:00           68241      21.6   
2 2017-05-01 12:00:00 2017-05-01 13:00:00           68241      23.5   
3 2017-05-01 13:00:00 2017-05-01 14:00:00           68241      23.0   
4 2017-05-01 14:00:00 2017-05-01 15:00:00           68241      24.1   

  MinT_Celsius MaxT_Celsius Precip_mm  
0         20.5         22.1       0.0  
1         21.6         23.5       0.0  
2         22.5         24.1       0.0  
3         22.7         24.7       0.0  
4         23.1         24.5       0.0  
8759
          valid_start           valid_end  station_number T_Celsius  \
0 2016-05-01 00:00:00 2016-05-01 01:00:00           68241      13.3   
1 2016-05-01 01:00:00 2016-05-01 02:00:00           68241      13.1   
2 2016-05-01 02:00:00 2016-05-01 03:00:00           68241      12.6   
3 2016-05-01 03:00:00 2016-05-01 04:00:

Step 5 : Merging dataframes and checking missing data

In [5]:
#a/ Merge dataframes

df3 = df2.append(df1, ignore_index=True)

print(len(df3)) #Check nb of lines after merge -> should be close to 8760*2= 17,520

#b/ Insert missing rows and drop rows outside year 2017

df3 = df3.resample('60Min', on='valid_start').first().drop('valid_start', 1).reset_index()
df4= df3.drop(df3[(df3.valid_start < "2017-01-01 00:00:00")|(df3.valid_start > "2017-12-31 23:00:00")].index)

if (len(df4))!=8760:
    print('Too many missing data, check your data')
    
print (df4.head()) #cross check data integrity
print (df4.tail())

#c/ Fill missing valid_end dates & station numbers

df4['valid_end']=df4['valid_start']+ datetime.timedelta(0,3600)
df4['station_number']=station

#d/ Check number of missing values and change format of empty data so they can be addressed at e/

print(len(df4))
df4 = df4.replace('', np.nan, regex=True)
print(df4.isna().sum())

#e/ Create a warning line each time a period of at least 5 consecutive days without data is identified

df4["T_Celsius"] = df4["T_Celsius"].apply(pd.to_numeric)
df4["MinT_Celsius"] = df4["MinT_Celsius"].apply(pd.to_numeric)
df4["MaxT_Celsius"] = df4["MaxT_Celsius"].apply(pd.to_numeric)
df4["Precip_mm"] = df4["Precip_mm"].apply(pd.to_numeric)
       
Datagap("T_Celsius",df4)
Datagap("MinT_Celsius",df4)
Datagap("MaxT_Celsius",df4)
Datagap("Precip_mm",df4)


17322
             valid_start           valid_end  station_number T_Celsius  \
5880 2017-01-01 00:00:00 2017-01-01 01:00:00         68241.0      21.5   
5881 2017-01-01 01:00:00 2017-01-01 02:00:00         68241.0      21.6   
5882 2017-01-01 02:00:00 2017-01-01 03:00:00         68241.0      21.5   
5883 2017-01-01 03:00:00 2017-01-01 04:00:00         68241.0      21.1   
5884 2017-01-01 04:00:00 2017-01-01 05:00:00         68241.0      21.0   

     MinT_Celsius MaxT_Celsius Precip_mm  
5880         21.5         21.8       0.0  
5881         21.4         21.6       0.0  
5882         21.1         21.5       0.0  
5883         20.9         21.2       0.2  
5884         20.9         21.1       0.0  
              valid_start valid_end  station_number T_Celsius MinT_Celsius  \
14635 2017-12-31 19:00:00       NaT             NaN       NaN          NaN   
14636 2017-12-31 20:00:00       NaT             NaN       NaN          NaN   
14637 2017-12-31 21:00:00       NaT             NaN      

Step 6 : Filling missing data and creating station csv file

In [6]:
#a/ Filling missing weather data based on x previous/following day(s)
    
Datafilling("T_Celsius",df4)
Datafilling("MinT_Celsius",df4)
Datafilling("MaxT_Celsius",df4)
Datafilling("Precip_mm",df4)

#b/ Final check and file export
                                                   
print(df4.isna().sum()) # Check nb of missing data
    
if (df4.isnull().values.any()):
    print ('Warning: datagap for a given hour > 5 days: check your data') # Warning if too many missing data at a given hour.
    
df4=df4.round({'T_Celsius': 1,'MinT_Celsius':1,'MaxT_Celsius':1,'Precip_mm':1}) # Final cleaning of decimals.
   
station=str(station)
df4.to_csv(r'csvfiles/weather2017_1H_'+station+'.csv',index=False)  # Export final csv file         
            

valid_start       0
valid_end         0
station_number    0
T_Celsius         0
MinT_Celsius      0
MaxT_Celsius      0
Precip_mm         0
dtype: int64
